<a href="https://colab.research.google.com/github/tomaszpasternak94/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import color, exposure

import os
import datetime

import tensorflow as tf

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout

from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin
#wizualizacja krzywych uczenia się
%load_ext tensorboard 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):    
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics= ['accuracy'])
 
 
  logdir = os.path.join('logs',datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq = 1)

  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size',128),
      epochs = params_fit.get('epochs',5),
      verbose = params_fit.get('verbose',1),
      validation_data = params_fit.get('validation_data',(X_train, y_train)),
      callbacks = [tensorboard_callback]
      
      )
  
  return model


def predict(model_trained, X_test, y_test, scoring = accuracy_score):
  
  y_test_norm = np.argmax(y_test, axis = 1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis = 1)

  return scoring( y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape,num_classes):
  return Sequential([

      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape), #warstwa konwolucyjna 
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'), 
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),      
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'), 
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),
      Conv2D(filters=128, kernel_size=(3,3), activation='relu'), 
      MaxPool2D(),
      Dropout(0.3),


      

      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(0.3),

      Dense(1024, activation='relu'),
      Dropout(0.3),

      Dense(num_classes, activation='softmax'),

      ])


#train_and_predict(get_cnn_v5(input_shape,num_classes))

In [32]:
model = get_cnn_v5(input_shape,num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 20s 74ms/step - loss: 2.6035 - accuracy: 0.2582 - val_loss: 1.2227 - val_accuracy: 0.5982
Epoch 2/5
272/272 [==============================] - 20s 73ms/step - loss: 0.7279 - accuracy: 0.7573 - val_loss: 0.1861 - val_accuracy: 0.9375
Epoch 3/5
272/272 [==============================] - 20s 73ms/step - loss: 0.2680 - accuracy: 0.9168 - val_loss: 0.0862 - val_accuracy: 0.9733
Epoch 4/5
272/272 [==============================] - 20s 73ms/step - loss: 0.1510 - accuracy: 0.9551 - val_loss: 0.0574 - val_accuracy: 0.9814
Epoch 5/5
272/272 [==============================] - 20s 73ms/step - loss: 0.1117 - accuracy: 0.9672 - val_loss: 0.0276 - val_accuracy: 0.9913


0.9607709750566893

In [11]:
model_trained.evaluate(X_test, y_test)[0]

138/138 [==============================] - 1s 7ms/step - loss: 0.0979 - accuracy: 0.9773


0.09787535667419434

In [0]:
def get_model(params):
  return Sequential([

      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape), #warstwa konwolucyjna 
      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'), 
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_one']),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'), 
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_two']),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'),
      Conv2D(filters=128, kernel_size=(3,3), activation='relu'), 
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_three']),


      

      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_one']),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_two']),

      Dense(num_classes, activation='softmax'),

      ])


#train_and_predict(get_cnn_v5(input_shape,num_classes))

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer = 'Adam', metrics= ['accuracy'])
  
  
  model.fit(
      X_train, 
      y_train, 
      batch_size = int(params.get('batch_size',128)), 
      epochs = 5,
      verbose = 0
      )
    
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
    
  return {'loss':-accuracy, 'status': STATUS_OK, 'model':model}


In [26]:
space = {
    'batch_size':hp.quniform('batch_size',100,200,10),
    'dropout_cnn_block_one' : hp.uniform('dropout_cnn_block_one',0.3, 0.5),
    'dropout_cnn_block_two' : hp.uniform('dropout_cnn_block_two',0.3, 0.5),
    'dropout_cnn_block_three' : hp.uniform('dropout_cnn_block_three',0.3, 0.5),

    'dropout_dense_block_one' : hp.uniform('dropout_dense_block_one',0.3, 0.7),
    'dropout_dense_block_two' : hp.uniform('dropout_dense_block_two',0.3, 0.7),


}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 140.0, 'dropout_cnn_block_one': 0.31718179263426266, 'dropout_cnn_block_three': 0.4635275441526284, 'dropout_cnn_block_two': 0.3471967240869176, 'dropout_dense_block_one': 0.5883392519545098, 'dropout_dense_block_two': 0.4119393108228791}
accuracy=0.9566893577575684
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.3041380161024805, 'dropout_cnn_block_three': 0.4784448863213524, 'dropout_cnn_block_two': 0.36351592778267333, 'dropout_dense_block_one': 0.3318777341234569, 'dropout_dense_block_two': 0.3437165789769888}
accuracy=0.9632652997970581
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.38688184628110217, 'dropout_cnn_block_three': 0.4296648759143866, 'dropout_cnn_block_two': 0.48478133292252523, 'dropout_dense_block_one': 0.3814959184799309, 'dropout_dense_block_two': 0.3931045989905098}
accuracy=0.9308390021324158
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.4845488800031574, 'dropout_cnn_block_three': 0.4438676700087306, 'dropout_cnn_block_two': 0.491458079761